In [ ]:
!pip install -q langchain-google-genai google-generativeai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# 🔑 Set your Gemini API Key here
os.environ["GOOGLE_API_KEY"] = ""

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
def ask_llm(prompt: str) -> str:
    resp = llm.invoke(prompt)   # just pass string
    return resp.content.strip()


In [ ]:
question = "Who discovered gravity and in which year?"

# Step 1: Draft
draft = ask_llm(f"Answer the question: {question}")
print("Draft:", draft)

# Step 2: Critique
critique = ask_llm(f"Here is a draft: {draft}\n\nCheck for mistakes and provide corrections.")
print("\nCritique:", critique)

# Step 3: Final
final = ask_llm(f"Draft: {draft}\nCritique: {critique}\n\nGive the corrected final answer.")
print("\nFinal Answer:", final)


Draft: Sir Isaac Newton is credited with formulating the law of universal gravitation, in 1687.  While others before him observed the effects of gravity, Newton was the first to mathematically describe it.

Critique: The draft is good but could be slightly improved for clarity and precision:

**Improved draft:**

> Sir Isaac Newton is credited with formulating the law of universal gravitation in 1687. While others before him observed the effects of gravity, Newton was the first to provide a comprehensive mathematical description of it.


**Changes and why:**

* **"in 1687" moved:**  It flows better grammatically to place the date immediately after the event it modifies.
* **"a comprehensive mathematical description" instead of "mathematically describe it":** This is more precise. Newton's contribution wasn't just a mathematical description, but a *comprehensive* one that explained a wide range of gravitational phenomena.  "Mathematically describe it" is a bit vague.


The original draf

In [ ]:
question = "Who discovered gravity and in which year?"

answer = ask_llm(f"Answer the question: {question}")
print("Initial Draft:", answer)

for i in range(3):  # run 3 reflection rounds
    reflection = ask_llm(
        f"Answer so far: {answer}\n\nReflect: what could be wrong or missing? "
        f"Correct and improve it."
    )
    answer = reflection
    print(f"\nAfter Reflection Round {i+1}:", answer)

print("\nFinal Answer after looped self-reflection:", answer)


Initial Draft: Sir Isaac Newton is credited with formulating the law of universal gravitation,  though the concept of gravity existed before him. He published his work on this in 1687, in his *Principia Mathematica*.

After Reflection Round 1: The answer is a good start, but it could be improved by adding nuance and precision:

**Improved Answer:**

Sir Isaac Newton is credited with formulating the *mathematical law* of universal gravitation, though the concept of gravity – the observation that objects fall to the earth – existed long before him.  He didn't discover gravity itself, but rather described its effects mathematically, proposing a universal law that explained both terrestrial and celestial motion.  He published this groundbreaking work in 1687 in his *Philosophiæ Naturalis Principia Mathematica* (*Mathematical Principles of Natural Philosophy*).  While his law provided an incredibly accurate model for many phenomena, it wasn't without limitations, later superseded by Einstei

In [ ]:
# Reflexion = multi-episode with memory of past reflections

memory = []

def evaluator(task, answer):
    """Very simple evaluator for demo."""
    if "newton" in answer.lower() and ("1687" in answer or "principia" in answer.lower()):
        return True, "PASS ✅"
    return False, "FAIL ❌ Missing Newton or year 1687"

def run_episode(task, memory):
    # Actor includes memory
    mem_text = "\n".join(memory) if memory else "No lessons yet."
    prompt = f"Lessons:\n{mem_text}\n\nTask: {task}\nAnswer clearly:"
    answer = ask_llm(prompt)

    # Evaluate
    success, feedback = evaluator(task, answer)

    # Reflection if fail
    if not success:
        reflection = ask_llm(
            f"Task: {task}\nAnswer: {answer}\nFeedback: {feedback}\n\n"
            f"Write one short lesson I should remember next time."
        )
        memory.append(reflection)

    return answer, success, feedback, memory

task = "Who discovered gravity and in which year?"

for ep in range(1, 4):  # 3 episodes
    print(f"\n--- Episode {ep} ---")
    ans, succ, fb, memory = run_episode(task, memory)
    print("Answer:", ans)
    print("Feedback:", fb)
    print("Memory:", memory)
    if succ:
        break



--- Episode 1 ---
Answer: Sir Isaac Newton described the law of universal gravitation in 1687.  While others before him observed the effects of gravity, Newton is credited with formulating the first scientific explanation.
Feedback: PASS ✅
Memory: []


# Auto-Verbalization Grading

In [9]:

!pip install -q langchain-google-genai google-generativeai


In [10]:

import os
from langchain_google_genai import ChatGoogleGenerativeAI

# 🔑 Set your Gemini API Key here
os.environ["GOOGLE_API_KEY"] = ""

# Initialize Gemini 1.5 Flash
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def ask_llm(prompt: str) -> str:
    """Simple helper to query Gemini."""
    resp = llm.invoke(prompt)
    return resp.content.strip()


In [11]:

def auto_verbalization_grader(question: str, student_answer: str, gold_answer: str = None):
    """
    Evaluates student answer by asking Gemini to explain correctness before grading.
    """
    prompt = f"""
You are a strict teacher grading a student's answer.

Question: {question}
Student Answer: {student_answer}
{"Correct Answer: " + gold_answer if gold_answer else ""}

Instructions:
1. First, explain in detail if the student answer is correct or not.
2. Then give a grade in this format:
Grade: Correct   OR   Grade: Incorrect
    """

    result = ask_llm(prompt)
    return result


In [13]:
# Colab cell 4 — Run a test example
question = "Who discovered gravity and in which year?"
student_answer = "Einstein discovered gravity in 1915."
gold_answer = "Isaac Newton discovered gravity in 1687."

graded_output = auto_verbalization_grader(question, student_answer, gold_answer)

print("=== Auto-Verbalization Grader Output ===\n")
print(graded_output)


=== Auto-Verbalization Grader Output ===

The student's answer is completely incorrect.  While Einstein's theory of General Relativity (1915) significantly advanced our understanding of gravity, describing it as a curvature of spacetime rather than a simple force, it did not *discover* gravity.  The discovery of gravity, as a fundamental force causing objects to attract each other, is attributed to Isaac Newton, who formulated the law of universal gravitation in 1687.  The student's answer confuses a refinement and expansion of the understanding of gravity with the initial discovery of the phenomenon itself.  Newton's work provided the foundational understanding of gravity that allowed for centuries of scientific progress before Einstein's revolutionary insights.

Grade: Incorrect


# Autogen Evaluator Prompt

In [18]:
# Initialize Gemini 1.5 Flash
assistant_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
evaluator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def ask_llm(llm, prompt: str) -> str:
    resp = llm.invoke(prompt)
    return resp.content.strip()

In [19]:

def assistant_agent(question: str) -> str:
    """Assistant tries to answer the user's question."""
    prompt = f"You are an assistant. Answer this question concisely:\n\n{question}"
    return ask_llm(assistant_llm, prompt)


In [20]:

def evaluator_agent(question: str, answer: str, gold: str = None) -> str:
    """Evaluator explains correctness (auto-verbalization) and gives grade."""
    prompt = f"""
You are an evaluator. Grade the assistant's answer.

Question: {question}
Assistant's Answer: {answer}
{"Correct Answer: " + gold if gold else ""}

Instructions:
1. First explain in detail if the answer is correct or not.
2. Then give a grade in this exact format:
Grade: Correct   OR   Grade: Incorrect
    """
    return ask_llm(evaluator_llm, prompt)


In [21]:

question = "Who discovered gravity and in which year?"
gold_answer = "Isaac Newton discovered gravity in 1687."

# Assistant generates an answer (it may be wrong or right)
assistant_output = assistant_agent(question)
print("=== Assistant Agent Output ===")
print(assistant_output)

# Evaluator checks the answer
evaluator_output = evaluator_agent(question, assistant_output, gold_answer)
print("\n=== Evaluator Sub-Agent Output ===")
print(evaluator_output)


=== Assistant Agent Output ===
Isaac Newton;  While the concept was understood earlier, Newton formulated the law of universal gravitation in the late 1660s, publishing it in 1687.

=== Evaluator Sub-Agent Output ===
The assistant's answer is partially correct but not entirely accurate.  While it correctly identifies Isaac Newton as the discoverer of the law of universal gravitation, the year is slightly off.  Newton's work on gravity culminated in the publication of *Principia Mathematica* in 1687, which is when his theory was formally presented to the world.  However, his work on the concept began much earlier, in the late 1660s.  The assistant's answer acknowledges this earlier work, but the question asks for the year of discovery, which is best represented by the year of publication of his seminal work.  Therefore, while the answer isn't completely wrong, it lacks precision in its response to the question's specific request for a single year.  The phrasing "While the concept was un